In [1]:
import os

In [2]:
%pwd

'd:\\Machine Learning Projects\\Kidney_disease_classification_project\\research'

In [3]:
os.chdir("D:\Machine Learning Projects\Kidney_disease_classification_project")

In [4]:
%pwd

'D:\\Machine Learning Projects\\Kidney_disease_classification_project'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [9]:
def id_extractor(url:str)->str:
    return url.split("/")[-2]

def load_dataset(file_id:str, path:Path)->None:
    prefix = "https://drive.google.com/uc?/export=download&id="
    gdown.download(prefix+file_id, path)
    

class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config
        
    def download_file(self)->str:
        '''
        fetch data from url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            load_dataset(id_extractor(url=dataset_url), path=zip_download_dir)
            logger.info(f"Data downloaded from {dataset_url} into file {zip_download_dir}")
            
        except Exception as e:
            raise e
        
    def extract_zip_file(self)->None:
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        function return None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-28 20:10:38,950] common, line:32 - INFO : yaml file: config\config.yaml loaded successfully
[2023-12-28 20:10:38,994] common, line:32 - INFO : yaml file: params.yaml loaded successfully
[2023-12-28 20:10:38,995] common, line:52 - INFO : created directory at: artifacts
[2023-12-28 20:10:38,997] common, line:52 - INFO : created directory at: artifacts/data_ingestion
[2023-12-28 20:10:39,002] 3935124273, line:21 - INFO : Downloading data from https://drive.google.com/file/d/18_Q2FRFj3MgVe0BjtNYR-Ppg1veBZ0-8/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=18_Q2FRFj3MgVe0BjtNYR-Ppg1veBZ0-8
From (redirected): https://drive.google.com/uc?/export=download&id=18_Q2FRFj3MgVe0BjtNYR-Ppg1veBZ0-8&confirm=t&uuid=b4c0c82c-ec02-49d7-8304-abefac6e1fd1
To: D:\Machine Learning Projects\Kidney_disease_classification_project\artifacts\data_ingestion\data.zip
100%|██████████| 941M/941M [22:07<00:00, 709kB/s]   

[2023-12-28 20:32:49,240] 3935124273, line:24 - INFO : Data downloaded from https://drive.google.com/file/d/18_Q2FRFj3MgVe0BjtNYR-Ppg1veBZ0-8/view?usp=sharing into file artifacts/data_ingestion/data.zip
